In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkml").getOrCreate()

In [2]:
spark

# Data1 Aggeragate

In [3]:
path0 = 's3://502pubg/Aggeragate/agg_match_stats_0.csv'
path1 = 's3://502pubg/Aggeragate/agg_match_stats_1.csv'
path2 = 's3://502pubg/Aggeragate/agg_match_stats_2.csv'
path3 = 's3://502pubg/Aggeragate/agg_match_stats_3.csv'
path4 = 's3://502pubg/Aggeragate/agg_match_stats_4.csv'

In [4]:
agg0 = spark.read.csv(path0, header=True, inferSchema=True)
agg1 = spark.read.csv(path1, header=True, inferSchema=True)
agg2 = spark.read.csv(path2, header=True, inferSchema=True)
agg3 = spark.read.csv(path3, header=True, inferSchema=True)
agg4 = spark.read.csv(path4, header=True, inferSchema=True)

In [5]:
agg = agg0.union(agg1)
agg = agg.union(agg2)
agg = agg.union(agg3)
agg = agg.union(agg4)
#Combine the csv files

In [6]:
agg.printSchema()

root
 |-- date: string (nullable = true)
 |-- game_size: integer (nullable = true)
 |-- match_id: string (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: double (nullable = true)
 |-- player_dist_walk: double (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_survive_time: double (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_placement: integer (nullable = true)



In [7]:
agg.show()

+--------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+--------------+-------------------+-------+--------------+
|                date|game_size|            match_id|match_mode|party_size|player_assists|player_dbno|  player_dist_ride|  player_dist_walk|player_dmg|player_kills|   player_name|player_survive_time|team_id|team_placement|
+--------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+--------------+-------------------+-------+--------------+
|2017-11-26T20:59:...|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|          2870.724|        1784.84778|       117|           1|      SnuffIes|            1106.32|      4|            18|
|2017-11-26T20:59:...|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|2938.4

In [8]:
agg=agg.drop('date')
agg =agg.drop('match_id')
agg = agg.drop('player_name')
agg = agg.drop('team_id')

In [9]:
agg.count()

67369231

In [10]:
agg = agg.na.drop()
#Drop the missing value

In [11]:
agg.count()

67369231

In [22]:
from pyspark.sql.types import IntegerType

In [23]:
agg0 = agg.withColumn('player_dist_ride',agg['player_dist_ride'].cast(IntegerType()))
agg0 = agg0.withColumn('player_dist_walk',agg['player_dist_walk'].cast(IntegerType()))
agg0 = agg0.withColumn('player_survive_time',agg['player_survive_time'].cast(IntegerType()))
#Change the type of some variables into integer.

In [42]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler,StandardScaler

In [25]:
indexer1 = StringIndexer(inputCol='match_mode', outputCol="matchmode_index")
indexer2 = StringIndexer(inputCol='team_placement', outputCol="label")
indexer3 = StringIndexer(inputCol='game_size', outputCol="gamesize_index")
indexer4 = StringIndexer(inputCol='party_size', outputCol="partysize_index")
#Transform qualitive variables into double type.

In [26]:
agg1 = indexer1.fit(agg0).transform(agg0)
agg1 = indexer2.fit(agg1).transform(agg1)
agg1 = indexer3.fit(agg1).transform(agg1)
agg1 = indexer4.fit(agg1).transform(agg1)

In [27]:
agg1.show()

+---------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-------------------+--------------+---------------+-----+--------------+---------------+
|game_size|match_mode|party_size|player_assists|player_dbno|player_dist_ride|player_dist_walk|player_dmg|player_kills|player_survive_time|team_placement|matchmode_index|label|gamesize_index|partysize_index|
+---------+----------+----------+--------------+-----------+----------------+----------------+----------+------------+-------------------+--------------+---------------+-----+--------------+---------------+
|       37|       tpp|         2|             0|          1|            2870|            1784|       117|           1|               1106|            18|            0.0| 15.0|          46.0|            1.0|
|       37|       tpp|         2|             0|          1|            2938|            1756|       127|           1|               1106|            18|            0.0| 15

In [28]:
agg1.describe().show()
#Summary the data.

+-------+-----------------+----------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+---------------+------------------+------------------+------------------+
|summary|        game_size|match_mode|        party_size|     player_assists|       player_dbno|  player_dist_ride|  player_dist_walk|        player_dmg|      player_kills|player_survive_time|    team_placement|matchmode_index|             label|    gamesize_index|   partysize_index|
+-------+-----------------+----------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+---------------+------------------+------------------+------------------+
|  count|         67369231|  67369231|          67369231|           67369231|          67369231|          67369231|          67369231|          6

In [30]:
agg1.createOrReplaceTempView('agg')

In [36]:
agg2 = spark.sql('select * FROM agg where agg.player_survive_time <5000')
#Remove the extrme value of player_survive_time

In [37]:
agg2.count()

67369186

In [38]:
vectorAssembler_features = VectorAssembler(
    inputCols=['player_assists','player_dbno','player_dist_ride','player_dist_walk','player_dmg','player_kills','player_survive_time','matchmode_index',"gamesize_index","partysize_index"], 
    outputCol="features",handleInvalid="skip")

In [39]:
agg3 = vectorAssembler_features.transform(agg2)
#Create the features vector

In [40]:
agg3.select('features').take(15)

[Row(features=DenseVector([0.0, 1.0, 2870.0, 1784.0, 117.0, 1.0, 1106.0, 0.0, 46.0, 1.0])),
 Row(features=DenseVector([0.0, 1.0, 2938.0, 1756.0, 127.0, 1.0, 1106.0, 0.0, 46.0, 1.0])),
 Row(features=SparseVector(10, {3: 224.0, 4: 67.0, 6: 235.0, 8: 46.0, 9: 1.0})),
 Row(features=SparseVector(10, {3: 92.0, 6: 197.0, 8: 46.0, 9: 1.0})),
 Row(features=DenseVector([0.0, 0.0, 2619.0, 2510.0, 175.0, 2.0, 1537.0, 0.0, 46.0, 1.0])),
 Row(features=DenseVector([0.0, 0.0, 2896.0, 2167.0, 65.0, 0.0, 1537.0, 0.0, 46.0, 1.0])),
 Row(features=SparseVector(10, {3: 3118.0, 6: 1264.0, 8: 46.0, 9: 1.0})),
 Row(features=SparseVector(10, {3: 3237.0, 6: 1264.0, 8: 46.0, 9: 1.0})),
 Row(features=SparseVector(10, {3: 1552.0, 4: 79.0, 6: 868.0, 8: 46.0, 9: 1.0})),
 Row(features=DenseVector([0.0, 1.0, 0.0, 1450.0, 101.0, 1.0, 868.0, 0.0, 46.0, 1.0])),
 Row(features=DenseVector([0.0, 1.0, 2406.0, 3100.0, 54.0, 1.0, 1939.0, 0.0, 46.0, 1.0])),
 Row(features=DenseVector([0.0, 0.0, 3135.0, 2456.0, 34.0, 0.0, 1939.0, 

In [44]:
scal = StandardScaler(inputCol='features',outputCol='stdfeatures',withStd=True, withMean=False)

In [45]:
agg4 = scal.fit(agg3).transform(agg3)
#Scale the features

In [46]:
agg4.printSchema()

root
 |-- game_size: integer (nullable = true)
 |-- match_mode: string (nullable = true)
 |-- party_size: integer (nullable = true)
 |-- player_assists: integer (nullable = true)
 |-- player_dbno: integer (nullable = true)
 |-- player_dist_ride: integer (nullable = true)
 |-- player_dist_walk: integer (nullable = true)
 |-- player_dmg: integer (nullable = true)
 |-- player_kills: integer (nullable = true)
 |-- player_survive_time: integer (nullable = true)
 |-- team_placement: integer (nullable = true)
 |-- matchmode_index: double (nullable = false)
 |-- label: double (nullable = false)
 |-- gamesize_index: double (nullable = false)
 |-- partysize_index: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- stdfeatures: vector (nullable = true)



In [47]:
agg4.select('stdfeatures').take(15)

[Row(stdfeatures=DenseVector([0.0, 0.8933, 1.4353, 0.4237, 0.6903, 0.6434, 1.9202, 0.0, 4.1005, 1.28])),
 Row(stdfeatures=DenseVector([0.0, 0.8933, 1.4693, 0.417, 0.7493, 0.6434, 1.9202, 0.0, 4.1005, 1.28])),
 Row(stdfeatures=SparseVector(10, {3: 0.0532, 4: 0.3953, 6: 0.408, 8: 4.1005, 9: 1.28})),
 Row(stdfeatures=SparseVector(10, {3: 0.0218, 6: 0.342, 8: 4.1005, 9: 1.28})),
 Row(stdfeatures=DenseVector([0.0, 0.0, 1.3098, 0.5961, 1.0325, 1.2867, 2.6684, 0.0, 4.1005, 1.28])),
 Row(stdfeatures=DenseVector([0.0, 0.0, 1.4483, 0.5146, 0.3835, 0.0, 2.6684, 0.0, 4.1005, 1.28])),
 Row(stdfeatures=SparseVector(10, {3: 0.7405, 6: 2.1945, 8: 4.1005, 9: 1.28})),
 Row(stdfeatures=SparseVector(10, {3: 0.7687, 6: 2.1945, 8: 4.1005, 9: 1.28})),
 Row(stdfeatures=SparseVector(10, {3: 0.3686, 4: 0.4661, 6: 1.507, 8: 4.1005, 9: 1.28})),
 Row(stdfeatures=DenseVector([0.0, 0.8933, 0.0, 0.3443, 0.5959, 0.6434, 1.507, 0.0, 4.1005, 1.28])),
 Row(stdfeatures=DenseVector([0.0, 0.8933, 1.2033, 0.7362, 0.3186, 0.6

In [48]:
agg4.write.parquet('s3://502pubg/clean/aggeragate3.parquet')

# DATA2 Deaths

In [50]:
path0 = 's3://502pubg/deaths/kill_match_stats_final_0.csv'
path1 = 's3://502pubg/deaths/kill_match_stats_final_1.csv'
path2 = 's3://502pubg/deaths/kill_match_stats_final_2.csv'
path3 = 's3://502pubg/deaths/kill_match_stats_final_3.csv'
path4 = 's3://502pubg/deaths/kill_match_stats_final_4.csv'

In [51]:
kill0 = spark.read.csv(path0, header=True, inferSchema=True)
kill1 = spark.read.csv(path1, header=True, inferSchema=True)
kill2 = spark.read.csv(path2, header=True, inferSchema=True)
kill3 = spark.read.csv(path3, header=True, inferSchema=True)
kill4 = spark.read.csv(path4, header=True, inferSchema=True)
#Read the data

In [52]:
kill0.count()

13426348

In [53]:
kill = kill0.union(kill1)
kill = kill.union(kill2)
kill = kill.union(kill3)
kill = kill.union(kill4)
#Combine the files

In [54]:
kill.printSchema()
#show the schema of the original data.

root
 |-- killed_by: string (nullable = true)
 |-- killer_name: string (nullable = true)
 |-- killer_placement: double (nullable = true)
 |-- killer_position_x: double (nullable = true)
 |-- killer_position_y: double (nullable = true)
 |-- map: string (nullable = true)
 |-- match_id: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- victim_name: string (nullable = true)
 |-- victim_placement: double (nullable = true)
 |-- victim_position_x: double (nullable = true)
 |-- victim_position_y: double (nullable = true)



In [55]:
kill.show()
#show the data

+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|   killed_by|     killer_name|killer_placement|killer_position_x|killer_position_y|    map|            match_id|time|    victim_name|victim_placement|victim_position_x|victim_position_y|
+------------+----------------+----------------+-----------------+-----------------+-------+--------------------+----+---------------+----------------+-----------------+-----------------+
|     Grenade| KrazyPortuguese|             5.0|         657725.1|         146275.2|MIRAMAR|2U4GBNA0YmnLSqvEy...| 823|KrazyPortuguese|             5.0|         657725.1|         146275.2|
|      SCAR-L|nide2Bxiaojiejie|            31.0|         93091.37|         722236.4|MIRAMAR|2U4GBNA0YmnLSqvEy...| 194|    X3evolution|            33.0|         92238.68|         723375.1|
|        S686|        Ascholes|            43.0|         366

In [56]:
kill.count()
# show the count of the data

65370475

In [57]:
kill0 = kill.na.drop()
# remove the missing value

In [58]:
kill0.count()

58923976

In [59]:
kill0 = kill0.drop('killer_name')
kill0 = kill0.drop('match_id')
kill0 = kill0.drop('victim_name')
#drop the unused variables

In [60]:
#Create a new feature that calculate the distance of killer and vic.
kill0.createOrReplaceTempView('kill0')

In [63]:
from pyspark.sql.functions import col,monotonically_increasing_id

In [61]:
x1=spark.sql('select Power((Power((killer_position_x-victim_position_x),2)+Power((killer_position_y-victim_position_y),2)),0.5) AS DIST from kill0').cache()

In [64]:
x1=x1.withColumn('id',monotonically_increasing_id())

In [66]:
kill0 = kill0.withColumn('id',monotonically_increasing_id())

In [67]:
kill1 = kill0.join(x1,kill0.id==x1.id,'inner')

In [68]:
kill1.printSchema()

root
 |-- killed_by: string (nullable = true)
 |-- killer_placement: double (nullable = true)
 |-- killer_position_x: double (nullable = true)
 |-- killer_position_y: double (nullable = true)
 |-- map: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- victim_placement: double (nullable = true)
 |-- victim_position_x: double (nullable = true)
 |-- victim_position_y: double (nullable = true)
 |-- id: long (nullable = false)
 |-- DIST: double (nullable = true)
 |-- id: long (nullable = false)



In [69]:
kill1 = kill1.drop('id')

In [70]:
kill1.show()

+------------+----------------+-----------------+-----------------+-------+----+----------------+-----------------+-----------------+------------------+
|   killed_by|killer_placement|killer_position_x|killer_position_y|    map|time|victim_placement|victim_position_x|victim_position_y|              DIST|
+------------+----------------+-----------------+-----------------+-------+----+----------------+-----------------+-----------------+------------------+
|       Punch|            31.0|         85697.73|         711625.3|MIRAMAR| 158|            33.0|         85646.66|         711693.0| 84.80232838780385|
|   Tommy Gun|            29.0|         432354.2|         552524.8|MIRAMAR| 175|            34.0|         432617.8|         552224.6| 399.5059448869687|
|       R1895|             5.0|         345684.9|         397896.6|ERANGEL| 125|            87.0|         344500.9|         396724.2|1666.2466083986242|
|        UMP9|            12.0|         645433.8|         320882.0|ERANGEL| 461|  

In [71]:
ind1 = StringIndexer(inputCol='killed_by',outputCol='weaponindex')
ind2 = StringIndexer(inputCol='map',outputCol='mapindex')

In [72]:
kill2 = ind1.fit(kill1).transform(kill1)
kill2 = ind2.fit(kill2).transform(kill2)
#Transfer the categorial data type.

In [73]:
vectorAssembler_features = VectorAssembler(
    inputCols=['killer_position_x','killer_position_y','victim_position_x','victim_position_y','DIST','weaponindex','mapindex'], 
    outputCol="features",handleInvalid="skip")

In [74]:
kill3 = vectorAssembler_features.transform(kill2)
# create the feature vector

In [75]:
kill3.show()

+------------+----------------+-----------------+-----------------+-------+----+----------------+-----------------+-----------------+------------------+-----------+--------+--------------------+
|   killed_by|killer_placement|killer_position_x|killer_position_y|    map|time|victim_placement|victim_position_x|victim_position_y|              DIST|weaponindex|mapindex|            features|
+------------+----------------+-----------------+-----------------+-------+----+----------------+-----------------+-----------------+------------------+-----------+--------+--------------------+
|       Punch|            31.0|         85697.73|         711625.3|MIRAMAR| 158|            33.0|         85646.66|         711693.0| 84.80232838780385|        8.0|     1.0|[85697.73,711625....|
|   Tommy Gun|            29.0|         432354.2|         552524.8|MIRAMAR| 175|            34.0|         432617.8|         552224.6| 399.5059448869687|       15.0|     1.0|[432354.2,552524....|
|       R1895|           

In [76]:
std2 = StandardScaler(inputCol='features',outputCol='stdfeatures')
#make the feature scale.

In [77]:
kill4 = std2.fit(kill3).transform(kill3)

In [78]:
kill4.select('stdfeatures').take(10)

[Row(stdfeatures=DenseVector([0.6012, 5.0881, 0.5675, 4.8261, 0.0009, 1.1151, 2.611])),
 Row(stdfeatures=DenseVector([3.033, 3.9505, 2.8666, 3.7447, 0.004, 2.0908, 2.611])),
 Row(stdfeatures=DenseVector([2.425, 2.8449, 2.2827, 2.6902, 0.0167, 3.4846, 0.0])),
 Row(stdfeatures=DenseVector([4.5278, 2.2943, 4.2805, 2.1756, 0.0056, 0.6969, 0.0])),
 Row(stdfeatures=DenseVector([4.4043, 2.7057, 4.1583, 2.5659, 0.0028, 0.4182, 0.0])),
 Row(stdfeatures=DenseVector([2.6415, 0.9889, 2.4945, 0.9376, 0.001, 1.1151, 0.0])),
 Row(stdfeatures=DenseVector([2.6011, 3.0275, 2.4551, 2.8755, 0.0068, 0.6969, 2.611])),
 Row(stdfeatures=DenseVector([3.2983, 4.5733, 3.1048, 4.3369, 0.0161, 1.3939, 0.0])),
 Row(stdfeatures=DenseVector([4.7568, 3.6569, 4.4934, 3.4689, 0.001, 0.5575, 0.0])),
 Row(stdfeatures=DenseVector([2.9759, 2.3349, 2.8102, 2.2085, 0.0088, 1.1151, 0.0]))]

In [79]:
kill4.printSchema()

root
 |-- killed_by: string (nullable = true)
 |-- killer_placement: double (nullable = true)
 |-- killer_position_x: double (nullable = true)
 |-- killer_position_y: double (nullable = true)
 |-- map: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- victim_placement: double (nullable = true)
 |-- victim_position_x: double (nullable = true)
 |-- victim_position_y: double (nullable = true)
 |-- DIST: double (nullable = true)
 |-- weaponindex: double (nullable = false)
 |-- mapindex: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- stdfeatures: vector (nullable = true)



In [80]:
kill4.write.parquet('s3://502pubg/clean/kill2.parquet')